In [12]:
import pandas as pd

In [13]:
msgs = pd.read_csv("SMSSpamCollection", sep = '\t', names = ['label', 'message'])

In [14]:
msgs.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [15]:
#Data cleaning and pre-processing

import re
import nltk


In [16]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [17]:
from nltk.stem import WordNetLemmatizer
wnlem = WordNetLemmatizer()

In [18]:
# for every message present in msgs object- replace all special characters with space using regex. 
# -convert uppercase to lowercase
# -tokenize using tokenize / split
# -lemmatize / stem if the word is not present in stopwords(english)


for i in range(0,len(msgs)):
    b = re.sub('\W', ' ', msgs['message'][i])
    b = b.lower()
    b = b.split()
    
    b = [ps.stem(word) for word in b if not word in set(stopwords.words('english'))]
    b = " ".join(b)
    
    corpus.append(b)

In [19]:
#creating the bag of words model

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000) #max_features = 2500
X = cv.fit_transform(corpus).toarray()

#if we do not specify max features, we get huge amount of features which might be redundant.
# by not specifying max_features, we have gotten 7252 words 
# by selecting max features as 5000, we get top 5000 frequent words

In [20]:
X.shape

(5572, 5000)

In [26]:
#encoding target variable "label"

y = pd.get_dummies(msgs['label'])
y = y.iloc[:,1].values # --1 
#get dummies creates two columns for each category with values 0s and 1s indicating 1 as present and 0 as absent. 
# so, you can drop one column(from --1) and keep only one column of the two 

In [29]:
# train-test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state= 0)

In [30]:
#training model using NB classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [36]:
y_pred = spam_detect_model.predict(X_test)

In [37]:
from sklearn.metrics import confusion_matrix

In [38]:
confusion_m = confusion_matrix(y_test, y_pred)

In [39]:
confusion_m

array([[948,   7],
       [  8, 152]], dtype=int64)

In [40]:
from sklearn.metrics import accuracy_score

In [41]:
accuracy = accuracy_score(y_test, y_pred)

In [42]:
accuracy

0.9865470852017937